---
title: "2023_Kronensicherung_Plesse_PTQ"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_PTQ
## ## PTQ: Daten der Elastometer (PicusTreeQinetic Data = PTQ)

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere das Paket treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Package Importe


Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend das extra geschriebenen Pakete PTQ. Fehler beim Import dieses Pakets sind ggf. Bugs. Es nutzte eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### Import von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

### Import TreeQinetic

Das Packet TreeQinetic wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der TXT-Messdaten der Picus TreeQinetic Elastometer und Inclinometer der Firma IML Instrumenta Mechanik Labor Electronic GmbH geschrieben (https://www.iml-electronic.de/produkt/picus-treeqinetic/). Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [2]:
import treeqinetic as ptq
# ptq.help() # Test

### Import von Latex Tabellen export Funktionen

In [3]:
from utils_latex_export import (
    save_latex_table,
    extract_latex_dict_from_json
)

### Projekt Konfiguration laden

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [4]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    main_path,
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)

## Daten Import

Aus dem Zugversuchsset wurden 4 Elastometer und 3 Inclinometer verwendet.

Die Elastometer waren auf dem rechten und linken Stämmling auf der Außenseite in zwei Ebenen platziert. Beim Zusammenziehen der Stämmlinge messen diese entsprechnd eine Faserdehnung, beim Ausschwingen der Stämmlinge über ihre Ruhelage hinaus nach außen eine Faserstauchung.

Die Inclinometer Daten werden hier ebefalls geladen, erschienen aber weniger geeignet zur Auswertung und werden entspricht nicht weiter berücksichtigt.

Die Funktion 'ptq.setup' erstellt div. Instanzen, die für das Paket notwendig sind (CONFIG, LOG_MANAGER, PLOT_MANAGER). 

Über die Klasse 'ptq.Series' wird eine neue Messreihe initialisiert und als 'ptq_series' gespeichert. Im Verzeichnis ptq_data_path finden sich die PTQ Daten als TXT von insgesamt 29 Messungen. Eine Datei enthält jeweils die Daten für alle Inclinometer und Elastometer.

In [5]:
ptq.setup(working_directory=working_directory, log_level="info", safe_logs_to_file=True)

ptq_data_path = data_path / 'PTQ/data_txt'
ptq_series = ptq.classes.Series(name=analyse_name, path=ptq_data_path)

elasto_names = ["Elasto(95)", "Elasto(98)", "Elasto(92)", "Elasto(90)"]

2025-03-23 12:08:05 [INFO] kj_logger.update_config: <kj_logger.LogManager object at 0x00000230BBE45190> initialized - update_config! Code: 000
2025-03-23 12:08:05 [WARNING] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ptq already exists.
2025-03-23 12:08:05 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ptq!
2025-03-23 12:08:05 [INFO] kj_core.core_config.__init__: <treeqinetic.config.Config object at 0x00000230BCF61750> initialized! Code: 001
2025-03-23 12:08:05 [INFO] treeqinetic.setup: ptq: Setup treeqinetic package!
2025-03-23 12:08:05 [INFO] kj_core.utils.plot_manager.__init__: <kj_core.utils.plot_manager.PlotManager object at 0x00000230BCB1B210> initialized! Code: 

## Explorative Datenanalyse

Übersicht über alle vom PTQ erfassten Daten über alle Messungen gemeinsam (Elastos und Inclinos).

In [6]:
ptq_df = ptq_series.get_measurements_df()
ptq_df.describe()

,ID,Nr,Datetime,Sec_Since_Start,Elasto(98),Elasto(95),Elasto(92),Elasto(90),Inclino(85)X,Inclino(85)Y,Inclino(83)X,Inclino(83)Y,Inclino(82)X,Inclino(82)Y,Inclino(81)X,Inclino(81)Y
count,17836.000000,17836.000000,17836,17836.000000,17461.000000,17468.000000,17657.000000,17676.000000,17448.000000,17448.0,17704.000000,17704.000000,17435.000000,17435.000000,17574.000000,17574.000000
mean,15.249047,338.457502,2023-03-22 13:34:17.235428864,85.133799,81.627536,105.786833,118.740437,191.147720,0.586046,0.0,0.735073,0.022071,-0.495272,-0.195005,0.009151,0.010782
min,1.000000,0.000000,2023-03-22 10:10:39.830198,0.000000,-192.900000,-119.100000,-111.000000,-217.800000,0.000000,0.0,-7.149000,-3.127000,-3.483000,-2.228000,-1.346000,-0.660000
25%,8.000000,153.000000,2023-03-22 11:11:37.444994048,38.984250,16.900000,8.400000,5.700000,13.475000,0.283000,0.0,0.424000,-0.102000,-0.922000,-0.397500,-0.006000,-0.005750
50%,16.000000,309.000000,2023-03-22 14:08:40.921139968,77.763500,95.100000,128.500000,139.500000,235.850000,0.557000,0.0,0.742000,0.000000,-0.522000,-0.221000,-0.002000,-0.001000
75%,23.000000,469.000000,2023-03-22 15:31:05.232380160,117.784500,137.500000,184.700000,213.100000,337.600000,0.848250,0.0,1.036000,0.127000,-0.070000,0.000000,0.001000,0.002000
max,29.000000,1392.000000,2023-03-22 16:12:55.455613,345.709000,234.600000,266.300000,291.500000,429.600000,7.029000,0.0,7.341000,4.525000,1.570000,1.147000,2.955000,0.526000
std,8.422199,242.404703,NaN,60.383660,64.555211,89.518611,101.978415,157.750503,0.392076,0.0,0.561636,0.253933,0.491294,0.271169,0.226831,0.075625


### Einhaltung der maximalen Faserdehnung

In [7]:
ptq_df_elasto_summary = ptq_df[elasto_names].describe()
ptq_df_elasto_summary

,Elasto(95),Elasto(98),Elasto(92),Elasto(90)
count,17468.000000,17461.000000,17657.000000,17676.000000
mean,105.786833,81.627536,118.740437,191.147720
std,89.518611,64.555211,101.978415,157.750503
min,-119.100000,-192.900000,-111.000000,-217.800000
25%,8.400000,16.900000,5.700000,13.475000
50%,128.500000,95.100000,139.500000,235.850000
75%,184.700000,137.500000,213.100000,337.600000
max,266.300000,234.600000,291.500000,429.600000


Analyse maximaler Dehnung und Stauchung, Prüfung auf Einhaltung der Elastizitätsgrenze (Fagus s. ca. 520 µm über 200~mm Länge des Elastos). 
Elasto(90) mit 429.6 mit der maximalen Faserdehnung

In [8]:
ptq_df_elasto_min_max = ptq_df_elasto_summary.loc[['min', 'max']]
ptq_df_elasto_min_max

,Elasto(95),Elasto(98),Elasto(92),Elasto(90)
min,-119.1,-192.9,-111.0,-217.8
max,266.3,234.6,291.5,429.6


In [9]:
latex_string = ptq_df_elasto_min_max.to_latex(index=True, escape=True, column_format="r|rrrr", float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Ergebnisse, Maximale Dehnungswerte der Elastometer"
caption_long = "Feldversuch 2 - Ergebnisse, Maximale Dehnungswerte der Elastometer in µm über die Länge der Messgeräte von 200~mm, jeweils der maximale gemessene Werte über alle Messungen, positive Werte = Faserdehnung, negative Werte = Faserstauchung"
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_maximale_dehnungswerte_der_elastometer.tex


Für jedes Elasto innerhalb jeder Messung getrennt. Werte werden bei der Gesamtauswertung später verwendet.

In [10]:
# Gruppiere nach ID und berechne min und max für jede Elasto-Spalte
ptq_df_elasto_min_max = ptq_df.groupby('ID')[elasto_names].agg(['min', 'max'])
ptq_df_elasto_min_max.head(3)

Elasto(95)        Elasto(98)        Elasto(92)        Elasto(90)       
          min    max        min    max        min    max        min    max
ID                                                                        
1      -119.1  215.0     -132.8  192.8      -86.6  240.9     -168.1  368.0
2       -90.4  254.4      -93.8  207.9     -111.0  282.6     -202.2  429.6
3      -106.8  262.2     -139.1  193.7      -87.0  291.5     -180.0  427.8

In [11]:
latex_string = ptq_df_elasto_min_max.reset_index().to_latex(index=False, escape=True, column_format="r|rr|rr|rr|rr", float_format="{:0.2f}".format)

caption = "Feldversuch 2 - Ergebnisse, Maximalwerte Elastometer vollständig"
caption_long = "Feldversuch 2 - Ergebnisse, Maximalwerte Elastometer vollständig, Faserstauchung als min., Faserdehnung als max. in µm über 200~mm Länge Messgerät"
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_maximalwerte_elastometer_vollstandig.tex


Die maximalen Dehnungs- und Stauchungswerte der Elastometer werden in der Struktur angepasst, um später mit den anderen Daten zusammenzuführen.

In [12]:
ptq_df_elasto = ptq_df_elasto_min_max.stack(level=0, future_stack=True).reset_index()

ptq_df_elasto = ptq_df_elasto.rename(columns={
    'ID': 'id', 
    'level_1': 'sensor_name',
    'max': 'max_strain',
    'min': 'max_compression'
})

ptq_df_elasto.head(10)

,id,sensor_name,max_compression,max_strain
0,1,Elasto(95),-119.1,215.0
1,1,Elasto(98),-132.8,192.8
2,1,Elasto(92),-86.6,240.9
3,1,Elasto(90),-168.1,368.0
4,2,Elasto(95),-90.4,254.4
5,2,Elasto(98),-93.8,207.9
6,2,Elasto(92),-111.0,282.6
7,2,Elasto(90),-202.2,429.6
8,3,Elasto(95),-106.8,262.2
9,3,Elasto(98),-139.1,193.7


## Plotten aller Messungen bzw. Elastometer

Plotten der 4 verwendeten Elastometer in einem Plot für jede Messung. Die Plots werden im Verzeichnis ptq/plots/multi_sensors_vs_time_1/ abgelegt.
Für alle Messungen und Elastometer ist gut zu erkennen, wie die Faserdehnung während des zusammen ziehen der Stämmlinge zunimmt, dann im Moment des Realises plötzlich abfällt, um in Folge harmonisch gedämpft auszuschwingen (nährungsweise).

In [13]:
ptq_series.plot_measurement_sensors(sensor_names=elasto_names)

2025-03-23 12:08:06 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' successful.
2025-03-23 12:08:07 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_101814.txt', ID: 2, Start '2023-03-22 10:18:33.334820' to '2023-03-22 10:21:25.691820'' successful.
2025-03-23 12:08:07 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_102314.txt', ID: 3, Start '2023-03-22 10:26:25.086505' to '2023-03-22 10:28:50.847505'' successful.
2025-03-23 12:08:07 [INFO] treeqinetic.classes.measurement.plot_multi_sensors: plot_multi_sensors for measurement: 'Measurement: 'PTQ_Meas_104400.txt', ID: 4, Start '2023-03-22 10:50:02.131987' to '2023-03-22 10:52:43.662987'' successful.
2025-03-23 12:08:07 [INFO] treeqinetic.class

## Bestimmung von Schwingungsparametern

### Selektieren der relevanten Bereiche

Selektiere die Bereiche nach dem Release, bei dem es zu einer harmonisch gedämpften Schwingung kommt. Die Methode Series.get_oscillations sucht im Standardfall nach einem Bereich in den Messdaten mit einer Länge von 20 Sekunden. Der Anfangszeitpunkt wird durch einen plötzlichen Abfall der Dehnung auf unter Null bestimmt, bei dem die Steigung mindestens -25 beträgt. Die Suche nach dem Startzeitpunkt beginnt erst 60 Sekunden nach Messungsbeginn. Der entsprechende Code befindet sich im Paket classes/measurement.py und utils/select_oscillation.py. Die so isolierten Bereiche werden als Instanzen der Klasse Oscillation initialisiert. Weitere Parameter wie Amplitude, Frequenz und Dämpfung werden direkt berechnet.
Parameter:
- sensor_names: Eine Liste der Sensornamen, für die die Schwingungsdaten identifiziert werden sollen.
- min_time_default: Die Mindestzeitspanne nach Beginn der Messung, nach der die Suche nach Schwingungen beginnt (Standard: 60 Sekunden).
- min_value: Der minimale Wertschwellenwert, damit Sensordaten als gültig betrachtet werden.
- threshold_slope: Der Steigungsschwellenwert, um den Beginn einer Schwingung zu bestimmen.
- duration: Die Dauer, für die die Schwingungsdaten extrahiert werden sollen.

In [14]:
ptq_series.get_oscillations(
    sensor_names=elasto_names,
    min_time_default=60,
    min_value=50,
    threshold_slope=-50,
    duration=17.5
)

2025-03-23 12:08:11 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'
2025-03-23 12:08:11 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_101814.txt', ID: 2, Start '2023-03-22 10:18:33.334820' to '2023-03-22 10:21:25.691820'
2025-03-23 12:08:11 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_102314.txt', ID: 3, Start '2023-03-22 10:26:25.086505' to '2023-03-22 10:28:50.847505'
2025-03-23 12:08:11 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_104400.txt', ID: 4, Start '2023-03-22 10:50:02.131987' to '2023-03-22 10:52:43.662987'
2025-03-23 12:08:11 [INFO] treeqinetic.classes.series.get_oscillations: 
 Select Oscillations for Measurement: 'PTQ_Meas_105549.txt', ID: 5, Start '2023-03-22 10:55:59.

### Selektion optisch prüfen in Plots

Plotten der relevanten Sensoren bzw. der selektierten Bereiche. Die Plots werden im Verzeichnis ptq/plots/select_oscillations_single/ bzw. ptq/plots/select_oscillations_combined/ gespeichert. In einem Combined-Plot werden alle 4 Elastometer einer Messung gemeinsam dargestellt. Hier wird manuell anhand der Plots geprüft, ob für alle Messungen und Sensoren der richtige Bereich ausgewählt wurde.

In [15]:
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=False)
ptq_series.plot_oscillations_for_measurements(sensor_names=elasto_names, combined=True)

2025-03-23 12:08:12 [INFO] treeqinetic.classes.series.plot_oscillations_for_measurements: Plot Oscillations for Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'
2025-03-23 12:08:12 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Elasto(95) successful.
2025-03-23 12:08:13 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Elasto(98) successful.
2025-03-23 12:08:13 [INFO] treeqinetic.classes.measurement._plot_single_oscillations: plot_select_oscillation for measurement: 'Measurement: 'PTQ_Meas_100346.txt', ID: 1, Start '2023-03-22 10:10:39.830198' to '2023-03-22 10:13:21.602198'' for Ela

### PTQ: Anpassen der harmonisch gedämpften Schwingung

Aus der PTQ-Messreihe 'ptq_series' wird über `get_oscillations_list` für alle Messungen, getrennt für jeden Sensor (Elastometer), die `Oscillation`-Instanz in eine Liste zusammengeführt. Entsprechend gibt es für jede PTQ-Messung 4 `Oscillation`-Instanzen (für die 4 Elastometer).

`oscillation.fit` passt alle Schwingungsdaten mit einer allgemeinen Funktion für harmonisch gedämpfte Schwingungen an:
\[
y(t) = A \cdot e^{-\delta t} \cdot \cos(\omega_d \cdot t + \phi) + y_0
\]

#### Parameterbeschreibung:
- `A` (Anfangsamplitude): Der Anfangswert der Amplitude der Schwingung. Dieser Parameter bestimmt die initiale Höhe der Schwingungsamplitude.
- `δ` (Dämpfungskoeffizient): Dieser Wert bestimmt, wie schnell die Amplitude der Schwingung mit der Zeit abnimmt. Ein höherer Wert führt zu einer schnelleren Dämpfung der Schwingung.
- `ω_d` (gedämpfte Kreisfrequenz): Die Frequenz der gedämpften Schwingung in Radiant pro Sekunde. Dieser Parameter bestimmt, wie schnell die Schwingung oszilliert.
- `φ` (Phasenwinkel): Der Anfangsphasenwinkel der Schwingung. Dieser Wert bestimmt den Startpunkt der Schwingung im Schwingungszyklus.
- `y_0` (Vertikale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve vertikal und ermöglicht es, die Schwingung an die mittlere Position der Daten anzupassen.
- `t_0` (Horizontale Verschiebung): Dieser Parameter verschiebt die gesamte Schwingungskurve horizontal über die Zeit und ermöglicht es, die Schwingung an den spezifischen Startpunkt der gemessenen Schwingung anzupassen.

(siehe `ptq/analyse/fitting_function.py`)

#### Zusätzliche Parameter und Konfigurationen:
- **Startwerte und Grenzwerte:** Für die Optimierung der Parameter in `scipy.curve_fit` werden Startwerte und Grenzwerte für jeden Parameter übergeben (in `ptq/config.py` definiert).
- **Qualitätsmetriken:** Zur Bewertung der Anpassungsgüte werden Metriken wie MAE (mittlerer absoluter Fehler), RMSE (Root Mean Square Error), und \( R^2 \) (Bestimmtheitsmaß) berechnet. Zusätzlich werden normalisierte Varianten (NRMSE und NMAE) zur besseren Vergleichbarkeit verwendet.
- **Warnungen bei Überschreitung der Grenzwerte:** Wenn die für eine Metrik definierten Grenzwerte überschritten werden, wird eine Warnung im Log-Protokoll vermerkt, um auf mögliche Probleme bei der Anpassung hinzuweisen (in `ptq/config.py` definiert). Auf Basis dieser Warnung können:
  - Start- und Grenzwerte sowie die Methodik angepasst werden.
  - Betroffene Datensätze später ausgeschlossen werden, um fehlerhafte Anpassungen zu vermeiden.
- **Interpolation:** Diese Option aktiviert die Interpolation der Datenpunkte, um eine ausreichende Dichte für `curve_fit` zu gewährleisten. Hierbei wird `scipy.interpolate.PchipInterpolator` verwendet, um Über- und Unterschwingungen, die nicht in den Originaldaten vorhanden sind, zu vermeiden. Nach optischer Prüfung zeigte diese Methode die besten Ergebnisse.

#### Visualisierungsoptionen:
- **Plot:** Wenn auf `True` gesetzt, wird für jede Oscillation ein Plot der angepassten Funktion zusammen mit den Originaldaten erstellt und in `working_dir/PTQ/plots/` gespeichert.
- **Plot-Fehlerverteilung:** Wenn `plot_error` auf `True` gesetzt ist, wird ein Histogramm der Fehlerverteilung (Residuen) für jeden Fit erstellt und ebenfalls in `working_dir/PTQ/plots/` gespeichert.

In [16]:
ptq_oscillations_ls = ptq_series.get_oscillations_list()

initial_param = {
    "initial_amplitude": 170,
    "damping_coeff": 0.32,
    "frequency": 0.44,
    "phase_angle": 0,
    "y_shift": 0,
    "x_shift": 0
}

param_bounds = {
    "initial_amplitude": (150, 250),
    "damping_coeff": (0.1, 1),
    "frequency": (0.35, 0.58),
    "phase_angle": (-0.2, 0.2),
    "y_shift": (-60, 60),
    "x_shift": (-0.25, 0.75),
}

metrics_warning = {
    "pearson_r": (0.75, 1),
    "nrmse": (0, np.inf),
    "mae": (0, np.inf),
    "nmae": (0, 0.10)
}

for oscillation in ptq_oscillations_ls:
    oscillation.fit(initial_param, param_bounds, optimize_criterion="mae", metrics_warning=metrics_warning, plot=True,
                    plot_error=True, dir_add="", interpolate=True)

2025-03-23 12:10:08 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(95)'' successful.
2025-03-23 12:10:11 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(98)'' successful.
2025-03-23 12:10:12 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(92)'' successful.
2025-03-23 12:10:15 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_100346.txt', ID: 1, Sensor: Elasto(90)'' successful.
2025-03-23 12:10:16 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_101814.txt', ID: 2, Sensor: Elasto(95)'' successful.
2025-03-23 12:10:18 [INFO] treeqinetic.classes.oscillation.fit: fit for measurement: 'Oscillation: 'PTQ_Meas_101814.txt', ID: 2, Sensor: Elasto(98)'' successful.
2025-03-23 12:10:21 [INFO] t

### Fehlerverteilung der Funktionsanpassung an Messdaten

Die Funktion sammelt für alle Oscillation-Objekte die Fehler-Arrays der Anpassung und normalisiert die Fehler (um Unterschiede in der Skalierung zu entfernen).
Anschließend werden die Fehler für alle Messungen A) für alle Sensoren gemeinsam und B) getrennt für jeden Sensor geplotet. Es werden Q-Q-Plot, Violin-Plot und Histogramme für den gleichen Sachverhalt erstellt und in `working_directory\PTQ\plots\series_osc_errors` abgelegt.

- `trim_hist_percent`: Beschneidet die Daten Links und Rechts um die äußersten x Prozent, da die Verteilung im Zentrum sonst kaum zu bewerten ist. Wirkt sich nur auf die Histogramme aus.

In [17]:
all_normalized_errors = ptq_series.plot_osc_errors(plot_qq=True, plot_violin=True, plot_hist=True, hist_trim_percent=5)

### PTQ: Zusammenfassung der Schwingungsparameter aller Oscillations als DataFrame

'ptq_series.get_oscillations_df' fasst aus allen Oscillation-Instanzen der Messreihe ('ptq_series') die Schwingungsparameter als pandas.DataFrame zusammen

In [18]:
ptq_metadata_df = ptq_series.get_oscillations_df()
# Converting 'sensor_name' to categorical
ptq_metadata_df['sensor_name'] = ptq_metadata_df['sensor_name'].astype('category')

metrics_warning_count = ptq_metadata_df['metrics_warning'].sum()
print(f"metrics_warning_count: {metrics_warning_count}")
ptq_metadata_df.head(3)

metrics_warning_count: 9


,id,file_name,sensor_name,sample_rate,max_value,min_value,m_amplitude,m_amplitude_2,metrics_warning,initial_amplitude,...,x_shift,pearson_r,p_value,r2,mse,rmse,nrmse,cv,mae,nmae
0,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,190.2,-119.1,154.65,82.25,False,170.297574,...,0.056037,0.932709,0.0,0.866563,220.603098,14.852713,0.048022,-0.520646,8.316242,0.026888
1,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,141.6,-132.8,137.20,92.50,False,170.139165,...,-0.071196,0.979672,0.0,0.955691,67.464479,8.213676,0.029934,-0.399424,5.589256,0.020370
2,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,239.9,-86.6,163.25,76.00,False,169.680935,...,-0.115989,0.931204,0.0,0.838016,177.244389,13.313316,0.040776,-1.212033,6.809497,0.020856


### PTQ: Berechnung ergänzender Schwingungsparameter

Ergänze rechnerische Schwingungsparameter die Rust(2013) genutzt hat

In [19]:
def calculate_additional_parameters(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate additional parameters (ungedämpfte Frequenz, Dämpfungsgrad)
    and append them to the DataFrame.

    Args:
    df (pd.DataFrame): Input DataFrame with necessary columns, where 'frequency' is in Hz.

    Returns:
    pd.DataFrame: DataFrame with additional parameters added.
    """
    # Erstelle eine Kopie des DataFrames, um inplace-Operationen zu vermeiden
    df = df.copy()

    # Umbenennen der Spalte 'frequency' zu 'frequency_damped'
    df = df.rename(columns={'frequency': 'frequency_damped'})

    # Berechnung der ungedämpften Frequenz
    df['frequency_undamped'] = np.sqrt(df['frequency_damped']**2 + (df['damping_coeff'] / (2 * np.pi))**2)

    # Berechnung des Dämpfungsgrades
    df['damping_ratio'] = df['damping_coeff'] / df['frequency_damped']

    return df

# Add additional parameters
ptq_metadata_df = calculate_additional_parameters(ptq_metadata_df)
ptq_metadata_df.head(5)


,id,file_name,sensor_name,sample_rate,max_value,min_value,m_amplitude,m_amplitude_2,metrics_warning,initial_amplitude,...,p_value,r2,mse,rmse,nrmse,cv,mae,nmae,frequency_undamped,damping_ratio
0,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,190.2,-119.1,154.65,82.25,False,170.297574,...,0.0,0.866563,220.603098,14.852713,0.048022,-0.520646,8.316242,0.026888,0.439638,0.647691
1,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,141.6,-132.8,137.20,92.50,False,170.139165,...,0.0,0.955691,67.464479,8.213676,0.029934,-0.399424,5.589256,0.020370,0.444416,0.540475
2,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,239.9,-86.6,163.25,76.00,False,169.680935,...,0.0,0.838016,177.244389,13.313316,0.040776,-1.212033,6.809497,0.020856,0.438912,0.643421
3,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,363.5,-168.1,265.80,162.10,False,171.136900,...,0.0,0.877885,594.489599,24.382157,0.045868,-60.280300,10.359435,0.019488,0.442028,0.365278
4,2,PTQ_Meas_101814.txt,Elasto(95),3.447691,226.9,-90.4,158.65,82.05,False,169.477340,...,0.0,0.806841,202.691266,14.236968,0.044873,-0.886650,7.701531,0.024274,0.426319,0.823806


## PTQ: Daten Vorbereitung (Data Cleaning)

Ergänze maximale Dehnung und Stauchung zum ptq_metadata_df

In [20]:
# Merging ptq_metadata_df and ptq_df_elasto
ptq_metadata_df = pd.merge(ptq_metadata_df, ptq_df_elasto, on=['id', 'sensor_name'], how='left')
ptq_metadata_df

,id,file_name,sensor_name,sample_rate,max_value,min_value,m_amplitude,m_amplitude_2,metrics_warning,initial_amplitude,...,mse,rmse,nrmse,cv,mae,nmae,frequency_undamped,damping_ratio,max_compression,max_strain
0,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,190.2,-119.1,154.65,82.25,False,170.297574,...,220.603098,14.852713,0.048022,-0.520646,8.316242,0.026888,0.439638,0.647691,-119.1,215.0
1,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,141.6,-132.8,137.20,92.50,False,170.139165,...,67.464479,8.213676,0.029934,-0.399424,5.589256,0.020370,0.444416,0.540475,-132.8,192.8
2,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,239.9,-86.6,163.25,76.00,False,169.680935,...,177.244389,13.313316,0.040776,-1.212033,6.809497,0.020856,0.438912,0.643421,-86.6,240.9
3,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,363.5,-168.1,265.80,162.10,False,171.136900,...,594.489599,24.382157,0.045868,-60.280300,10.359435,0.019488,0.442028,0.365278,-168.1,368.0
4,2,PTQ_Meas_101814.txt,Elasto(95),3.447691,226.9,-90.4,158.65,82.05,False,169.477340,...,202.691266,14.236968,0.044873,-0.886650,7.701531,0.024274,0.426319,0.823806,-90.4,254.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,28,PTQ_Meas_155805.txt,Elasto(90),2.392208,322.6,-95.4,209.00,97.65,False,169.647929,...,337.379431,18.367891,0.043943,0.937286,10.239514,0.024497,0.528304,0.723144,-95.4,323.2
112,29,PTQ_Meas_160907.txt,Elasto(95),2.967190,228.1,-23.9,126.00,46.20,False,169.564768,...,410.272163,20.255176,0.080378,3.804639,9.617778,0.038166,0.455556,0.777837,-23.9,237.6
113,29,PTQ_Meas_160907.txt,Elasto(98),2.967190,150.4,-72.7,111.55,89.70,False,169.967223,...,439.556078,20.965593,0.093978,-9.177909,12.110682,0.054286,0.456087,0.468520,-72.7,165.8
114,29,PTQ_Meas_160907.txt,Elasto(92),2.967190,259.7,-22.0,140.85,39.30,False,169.737537,...,488.029395,22.091387,0.078422,5.346908,10.226893,0.036304,0.463280,0.994797,-28.3,263.4


In [ ]:
# Liste der Spalten, die beibehalten werden sollen
select_cols = [
    'id', 
    'sensor_name',
    'max_strain',
    'max_compression',
    'm_amplitude', 
    'm_amplitude_2', 
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped', 
    'y_shift',
    'pearson_r',
    'nrmse', 
    'nmae', 
]

# DataFrame auf die gewünschten Spalten reduzieren
ptq_metadata_df = ptq_metadata_df[select_cols]


In [ ]:
ptq_metadata_df.head(3)

#### Datendokumentation:

In [ ]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "feature_data_dict.json", "r", encoding="utf-8") as f:
    feature_data_dict = json.load(f)

data_dict = {**sensor_position_dict, **series_dict, **feature_data_dict}

# Keys aller gewünschten Variablen (aus kombiniertem Dict)
selected_keys = ["id", "treatment", "sensor_name", "location", "height", "diameter", "rope_release", "max_strain", "calc_max_strain", "strain_difference", "release_force_target", "cable_max", "max_compression", "m_amplitude", "m_amplitude_2", "initial_amplitude", "damping_coeff", "damping_ratio", "frequency_damped", "frequency_undamped", "y_shift", "pearson_r", "nmae"]

# Nutze bestehende Felder
selected_fields = ["Zeichen", "Variable", "Deutsch", "Einheit", "Beschreibung"]

# Erzeuge DataFrame für LaTeX mit einheitlichem Aufbau
combined_dict_df = extract_latex_dict_from_json(data_dict, selected_keys, selected_fields)

# Exportiere als LaTeX
latex_string = combined_dict_df.to_latex(index=False, escape=False)

caption = "Feldversuch 2 - Ergebnisse, Daten Dokumentation"
caption_long = "Feldversuch 2 - Ergebnisse, Kräfte, Dehnungen und Schwingungsparameter, Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)